## 1. Tensor

 - 텐서플로우를 구성하는 가장 기초적인 데이터 단위
 - n 차원 배열의 집합 또는 n 차원 배열을 의미
 - rank: 텐서의 차원
 - 텐서의 표현은 `numpy` 배열을 사용

`3. # a rank 0 tensor; a scalar with shape []`  
`[1., 2., 3.] # a rank 1 tensor; a vector with shape [3],`  
`[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3],`  
`[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]`  

## 2. TensorFlow programming

 #### 1) 연산 그래프(computational graph) 정의
  - 연산그래프는 텐서플로우 작업을 순차적으로 정의(표현)한 것으로 노드와 에지를 갖는 그래프 형태를 갖음 
  - 연산그래프의 노드에는 텐서를 입력값으로 받아 연산하는 작업들이 위치 : `tf.Operation`
  - 연산그래프의 에지에는 노드에 정의된 연산간에 주고 받는 데이터 들을 표현(텐서들이 그래프 상에서 흐름.) `tf.Tensor`

 #### 2) 연산 그래프를 실행
  - 연산그래프의 실행은 `tf.Session` 객체,텐서플로우가 실행되는 환경을 만들어서 진행됨
  - 연산그래프의 작업을 CPU, GPU에 배정하고 실행을 위한 메서드를 제공

#### [default 그래프에 정의하기]
 - 3개의 노드(2개: constant op, 1개 matmul op)
 - 특정 그래프 객체에 명시적으로 연산을 정의하지 않는한 모든 연산은 전역 default 그래프에 정의됨 

<img src="./images/tf_graph_1.png" alt="graph_1" width="700" align="left"/>

In [ ]:
import tensorflow as tf

In [ ]:
mat_a = tf.constant([[3.0, 3.0]], dtype=tf.float32)
mat_b = tf.constant([[2.0],
                     [2.0]], dtype=tf.float32)
product = tf.matmul(mat_a, mat_b)

print(tf.get_default_graph() is product.graph)

#### [특정 그래프에 연산 정의하기]

In [ ]:
g_1 = tf.Graph()
g_2 = tf.Graph()

with g_1.as_default():
    mat_a = tf.constant([[3.0, 3.0]], dtype=tf.float32)
    mat_b = tf.constant([[2.0],
                         [2.0]], dtype=tf.float32)
    product1 = tf.matmul(mat_a, mat_b)
    print(product1.graph is g_1)
    print(product1.graph is g_2)

with g_2.as_default():
    mat_c = tf.constant([[3.0, 3.0]], dtype=tf.float32)
    mat_d = tf.constant([[2.0],
                         [2.0]], dtype=tf.float32)
    product2 = tf.matmul(mat_c, mat_d)
    print(product2.graph is g_1)
    print(product2.graph is g_2)

#### [그래프 실행하기]
 - session 객체의 run 매서드 호출
 - default 그래프에 정의한 3개의 작업이 실행 (graph=None)
 - 사용한 session 반환

In [ ]:
sess = tf.Session(graph=g_2)
print(sess.run(product2))
sess.close()

In [ ]:
sess = tf.Session(graph=g_1)
print(sess.run(product2))
sess.close()

 - session 컨텍스트 매니저 활용

In [ ]:
with tf.Session(graph=g_2) as sess:
    print(sess.run(product2))

## 3. TensorFlow tf.constant, tf.Variable

 - 연산 그래프에 정의된 연산을 수행하기 위해 필요한 데이터 값을 입력 위한 수단 

 ### 3-1. tf.constant
 - 상수 텐서를 생성하는 작업으로, `tf.constant` 연산 정의시 제공한 초기값을 갖는 텐서를 반환

 ### 3-2. tf.Variable
 - 텐서플로우 프로그램에서 연산의 결과가 공유되고, 상태를 유지해야하는 경우 사용 
   - ex) 학습을 진행하면서 모델의 파라미터가 업데이트 되야하므로 모델의 파라미터를 변수로 표현
 - 변수 연산을 정의하기 위해 텐서를 초기값으로 부여, 초기값으로 제공한 텐서로 변수 type과 shape이 결정됨
 - 변수 연산이 정의되면 타입과 변수 type과 shape은 고정됨, 변수 값인 텐서를 assign 메서드로 변경
 - 연산을 실행하기 전, 그래프 상에 정의된 변수를 명시적으로 초기화하는 작업 필요
   - 초기화 연산을 실행(`tf.global_variable_initializer()`), 변수 값이 저장된 파일에서 복구, `assign` 메서드 실행

<img src="./images/tf_graph_2.png" alt="graph_1" width="700" align="left"/>

In [ ]:
tf.reset_default_graph()

state = tf.Variable(0, name="counter")
one = tf.constant(1)

new_value = tf.add(state, one)
update = tf.assign(state, new_value)

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init_op)
    
    print(sess.run(state))
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

 #### [변수의 저장과 복구]
  - 변수를 이름과 텐서 값을 매핑해놓은 바이너리 파일(`.ckpt`)에 저장 가능
  - `tf.train.Saver()` 객체를 이용하여 그래프 전체 변수와 지정된 리스트 변수를 저장하고 복구
  - 저장될 때 사용되는 변수 명은 `Variable.name`이 기본 값
  - `tf.train.Saver()` 객체에 딕셔너리를 저장할 이름(key), 저장할 값(value)로 전달하여 저장시 사용할 이름을 변경하거나 변수를 선택적으로 저장 가능
    - ex) `tf.train.Saver({"saved_v":v})`
  - 전체 변수를 파일에서 복구 시 변수 초기화가 필요 없음

In [ ]:
# Create some variables.
import tensorflow as tf
import os
tf.reset_default_graph()

v1 = tf.Variable(tf.zeros([3]), name="v1")
v2 = tf.Variable(tf.zeros([3]), name="v2")
    
inc_v1 = v1.assign(v1+1)
dec_v2 = v2.assign(v2-1)

init_op = tf.global_variables_initializer()

saver = tf.train.Saver()

with tf.Session() as sess:
    
    sess.run(init_op)
    print("Values to be saved:", sess.run([inc_v1, dec_v2]))
    
    os.makedirs("./tmp/ckpt", exist_ok=True)
    save_path = saver.save(sess, "./tmp/ckpt/model.ckpt")
    print("Model saved in path: %s" % save_path)
    
    
from tensorflow.python.tools import inspect_checkpoint as chkp
chkp.print_tensors_in_checkpoint_file(save_path, tensor_name='',  all_tensors=True)

In [ ]:
tf.reset_default_graph()

v1 = tf.Variable(tf.zeros([3]), name="v1")
v2 = tf.Variable(tf.zeros([3]), name="v2")

saver = tf.train.Saver()

with tf.Session() as sess:

    saver.restore(sess, "./tmp/ckpt/model.ckpt")
    print("Model restored.")
    print("v1 : %s" % v1.eval())
    print("v2 : %s" % v2.eval())

## 4. Fetches

 - 그래프 상에 정의된 작업 하나 이상의 작업 실행 결과 가져오기

In [ ]:
import tensorflow as tf
tf.reset_default_graph()

input1 = tf.constant([3.0])
input2 = tf.constant([2.0])
input3 = tf.constant([5.0])

intermed = tf.add(input2, input3)
mul = tf.multiply(input1, intermed)

with tf.Session() as sess:
    result = sess.run([mul, intermed])
    print(result)

## 5. Feed

 - 실행 시점에 정의된 연산 그래프 상으로 텐서 값을 제공하는 매커니즘
 - tf.placeholder를 이용 텐서(데이터)가 연산 그래프에 입력될 입력 공간을 확보
 
 <img src="./images/tf_graph_3.png" alt="graph_3" width="700" align="left"/>

In [ ]:
import numpy as np
tf.reset_default_graph()

x1 = tf.placeholder(tf.float32, shape=(4, 4), name='input1')
x2 = tf.placeholder(tf.float32, shape=(4, 4), name='input2')
y = tf.matmul(x1, x2)

with tf.Session() as sess:
    arr1 = np.array([
        [1,2,3,4],
        [2,3,4,5],
        [3,4,5,6],
        [4,5,6,7]
    ])
    arr2 = np.array([
        [2,3,4,5],
        [3,4,5,6],
        [4,5,6,7],
        [5,6,7,8]
    ])
    print(sess.run(y, feed_dict={x1: arr1, x2:arr2}))

In [ ]:
print(1*2 + 2*3 + 3*4 + 4*5)
print(1*5 + 2*6 + 3*7 + 4*8)
print(4*2 + 5*3 + 6*4 + 7*5)
print(4*5 + 5*6 + 6*7 + 7*8)